In [2]:
import util
import importlib
import numpy as np
import torch

# Read sentences from tsv files

In [3]:
importlib.reload(util)
df = util.get_dataframe(lang_pairs=['et-en', 'ro-en', 'ne-en'], is_dev_sentence=True)

print(df)


Processing et-en/dev.eten.df.short.tsv ...
Processing ro-en/dev.roen.df.short.tsv ...
Processing ne-en/dev.neen.df.short.tsv ...
      level_0  index                                           original  \
0           0      0  Kaks sõdurit sai vigastada ja toimetati ravile...   
1           1      1  Kant ütleb, et kõik käsitööd, tööndused ja kun...   
2           2      2  Ülestõus kestis 1956. aasta 23. oktoobrist 10....   
3           3      3  Religioon pakub vaimu puhastamiseks teatud vah...   
4           4      4  Duns Scotus hakkas Pariisis "Sententiae" üle l...   
...       ...    ...                                                ...   
2995      995    995  नयाँ संविधान जारी भएपछि सरकार र राष्ट्रपतिलाई ...   
2996      996    996  कूर्मपूराणको यो श्लोकले पनि परापूर्वकालदेखि नै...   
2997      997    997  प्रतियोगितामा २ गोल र २ गोल सहायता प्रदान गरेक...   
2998      998    998  स्पेनमा रोमन क्याथोलिक चर्चहरुको कडा विरोधको व...   
2999      999    999  एउटा पक्षले बृहत हिमालय 

In [4]:
print(df.loc[900])
print()
print(df.loc[1800])
print()
print(df.loc[2700])

level_0                                                       900
index                                                         900
original        Galileo Galilei vaatles oma teleskoobiga Kuud ...
translation     Galileo looked at Cuba with its television cod...
scores                                       [5, 31, 1, 8, 1, 31]
mean                                                    12.833333
z_scores        [-1.4915355021939918, -0.6798210592198779, -1....
z_mean                                                  -1.492266
model_scores                                            -0.406991
Name: 900, dtype: object

level_0                                                       800
index                                                         800
original        Unul dintre revoluționari, Louis Blanc, era cu...
translation     One of the revolutionaries, Louis Blanc, was k...
scores                               [100, 100, 91, 100, 100, 85]
mean                                              

In [5]:
# Max length sentence
print(df['original'].map(len).max())
print(df['translation'].map(len).max())

print(df['original'].map(len).argmax())
print(df['translation'].map(len).argmax())

150
628
354
2379


In [6]:
df.loc[2379]['translation']

'Of the growth of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the descendants of the'

# Tokenize sentences

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


In [8]:
target_sentences = df['translation'].values.tolist()
source_sentences = df['original'].values.tolist()

result = tokenizer(target_sentences, source_sentences, padding='max_length', return_tensors='pt')

input_ids = result['input_ids']
token_type_ids = result['token_type_ids']
attention_mask = result['attention_mask']


In [9]:
print(input_ids.shape)
print(token_type_ids.shape)
print(attention_mask.shape)

torch.Size([3000, 512])
torch.Size([3000, 512])
torch.Size([3000, 512])


In [10]:
tokenizer.decode(result['input_ids'][0])

"[CLS] Two soldiers were injured and brought to Tskinvali's hospital for treatment. [SEP] Kaks sõdurit sai vigastada ja toimetati ravile Tshinvali haiglasse. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

# Run BERT model

In [18]:
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
result = bert_model(input_ids[0:10, :], attention_mask[0:10, :], token_type_ids[0:10, :])

print(result['last_hidden_state'].shape)
print(result['pooler_output'].shape)
print()
print(result['last_hidden_state'])

torch.Size([10, 512, 768])
torch.Size([10, 768])

tensor([[[-1.3462e-01,  2.0204e-01, -5.2373e-02,  ...,  1.8463e-01,
           5.1738e-02, -6.1155e-02],
         [-3.9784e-01, -4.4938e-01,  6.4302e-01,  ...,  5.1545e-01,
           1.9946e-01, -3.9049e-01],
         [-5.4147e-01,  8.5260e-02,  5.8891e-01,  ...,  2.7444e-01,
          -2.0853e-01, -2.9998e-01],
         ...,
         [-1.0484e-01, -9.2726e-02,  2.7370e-01,  ...,  2.1666e-01,
           3.6425e-01,  2.0545e-01],
         [-1.4444e-01, -1.2643e-01,  2.7037e-01,  ...,  2.0216e-01,
           3.8089e-01,  1.5077e-01],
         [-2.3956e-01, -4.7839e-02,  3.8554e-01,  ..., -1.3457e-01,
           4.1152e-01,  1.3971e-02]],

        [[-1.9649e-02,  1.5470e-01,  1.4945e-01,  ...,  8.7340e-02,
           3.1913e-03, -2.3092e-01],
         [-2.9002e-01,  2.9691e-01,  3.4906e-01,  ...,  4.3705e-01,
          -1.7082e-01, -5.3481e-01],
         [-3.2392e-01,  2.6195e-01,  3.5419e-01,  ...,  4.3900e-01,
           4.9521e-01, -5.